# Estudo sobre as eleições municipais de 2020 e 2024 no estado de Minas Gerais

## Instalando e importando os pacotes necessários

In [5]:
!pip install plotly geopandas matplotlib pandas

In [50]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px

## Definindo arquivos necessários e váriaveis globais 

In [9]:
# Caminho para os arquivos utilizados
pasta = 'C:\\Users\\marce\\OneDrive\\Área de Trabalho\\analise_eleitoral\\'
arquivo_resultado_2020 = os.path.join(pasta, 'votacao_2020.csv')
arquivo_resultado_2024 = os.path.join(pasta, 'votacao_2024.csv')
arquivo_partidos = os.path.join(pasta,'partidos.csv')

In [13]:
# Alterando formato de exibição dos gráficos do Pandas
pd.options.plotting.backend = 'plotly'

In [12]:
# Variáveis globais
TIPO_CANDIDATO = 'Prefeito' #Recebe Vereador pu Prefeito

In [25]:
# Lendo os dados dos arquivos
tabela_arquivo_2020 = pd.read_csv(arquivo_resultado_2020, sep=';', encoding = "ISO-8859-1")
tabela_arquivo_2024 = pd.read_csv(arquivo_resultado_2024, sep=';', encoding = "ISO-8859-1")
tabela_partidos = pd.read_csv(arquivo_partidos, sep=';')

In [28]:
tabela_arquivo_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10411 entries, 0 to 10410
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   DT_GERACAO                      10411 non-null  object
 1   HH_GERACAO                      10411 non-null  object
 2   ANO_ELEICAO                     10411 non-null  int64 
 3   CD_TIPO_ELEICAO                 10411 non-null  int64 
 4   NM_TIPO_ELEICAO                 10411 non-null  object
 5   NR_TURNO                        10411 non-null  int64 
 6   CD_ELEICAO                      10411 non-null  int64 
 7   DS_ELEICAO                      10411 non-null  object
 8   DT_ELEICAO                      10411 non-null  object
 9   TP_ABRANGENCIA_ELEICAO          10411 non-null  object
 10  SG_UF                           10411 non-null  object
 11  SG_UE                           10411 non-null  int64 
 12  NM_UE                           10411 non-null

In [77]:
# Funções

def processar_candidatos_mais_votados(tabela_arquivo):
    return tabela_arquivo\
        .sort_values(by='NM_UE', ascending=False)\
        .groupby(['NM_PARTIDO', 'NM_UE'])\
        .agg({'QT_VOTOS_NOMINAIS_VALIDOS': 'first', 'NR_PARTIDO': 'first'})\
        .reset_index()

def gerar_grafico_partidos_mais_votados(df, ano):
    # Agrupar os dados por partido e contar o número de municípios em que cada partido venceu
    votos_por_partido = df.groupby('NM_PARTIDO')['QT_VOTOS_NOMINAIS_VALIDOS'].count().reset_index()
    votos_por_partido = votos_por_partido.sort_values(by='QT_VOTOS_NOMINAIS_VALIDOS', ascending=False)

    # Gerar o gráfico de barras
    fig = px.bar(votos_por_partido, 
                 x='NM_PARTIDO', 
                 y='QT_VOTOS_NOMINAIS_VALIDOS', 
                 title=f'Partidos mais votados em {ano}',
                 labels={'QT_VOTOS_NOMINAIS_VALIDOS': 'Número de Municípios', 'NM_PARTIDO': 'Partido'},
                 color='NM_PARTIDO')

    fig.show()

In [78]:
candidatos_mais_votados_2020 = processar_candidatos_mais_votados(tabela_arquivo_2020)

In [79]:
display(candidatos_mais_votados_2020)

,NM_PARTIDO,NM_UE,QT_VOTOS_NOMINAIS_VALIDOS,NR_PARTIDO
0,AVANTE,AIMORÉS,627,70
1,AVANTE,ALBERTINA,456,70
2,AVANTE,ALFENAS,1953,70
3,AVANTE,ALFREDO VASCONCELOS,672,70
4,AVANTE,ALTEROSA,2707,70
...,...,...,...,...
6264,Solidariedade,ÁGUAS VERMELHAS,76,77
6265,Unidade Popular,BELO HORIZONTE,726,80
6266,Unidade Popular,ITABIRITO,477,80
6267,Unidade Popular,NOVA LIMA,764,80


In [80]:
candidatos_mais_votados_2024 = processar_candidatos_mais_votados(tabela_arquivo_2024)

In [81]:
display(candidatos_mais_votados_2024)

,NM_PARTIDO,NM_UE,QT_VOTOS_NOMINAIS_VALIDOS,NR_PARTIDO
0,AGIR,ALPINÓPOLIS,752,36
1,AGIR,ALVINÓPOLIS,758,36
2,AGIR,ANDRADAS,1186,36
3,AGIR,ARAXÁ,2387,36
4,AGIR,ARCOS,2288,36
...,...,...,...,...
8362,União Brasil,VIRGINÓPOLIS,526,44
8363,União Brasil,VIRGÍNIA,704,44
8364,União Brasil,VISCONDE DO RIO BRANCO,2103,44
8365,União Brasil,VIÇOSA,16346,44


## Gráficos dos partidos mais votados nos anos de 2020 e 2024

In [82]:
gerar_grafico_partidos_mais_votados(candidatos_mais_votados_2020, 2020)

In [83]:
gerar_grafico_partidos_mais_votados(candidatos_mais_votados_2024, 2024)

## Unindo as tabelas dos candidatos e dos partidos
### Assim podemos definir quais candidatos são de direita, esquerda ou centro

In [84]:
espec_type = pd.CategoricalDtype(categories=['direita', 'centro', 'esquerda'], ordered=True)
tabela_partidos['Espectro'] = tabela_partidos['Espectro'].astype(espec_type)

In [85]:
candidatos_mais_votados_2020 = candidatos_mais_votados_2020.\
merge(tabela_partidos[['NR_PARTIDO','Espectro']], left_on='NR_PARTIDO', right_on='NR_PARTIDO')

In [87]:
candidatos_mais_votados_2024 = candidatos_mais_votados_2024.\
merge(tabela_partidos[['NR_PARTIDO','Espectro']], left_on='NR_PARTIDO', right_on='NR_PARTIDO')

## Quem venceu mais: Esquerda ou Direita?
